<a href="https://colab.research.google.com/github/Computer-CGuy/etherpaisahipaisa/blob/master/Soil_Sample_%7C_JaxEdition.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import jax
import numpy as np

In [2]:
FNV_PRIME = 0x01000193

def fnv(v1, v2):
    return ((v1 * FNV_PRIME) ^ v2) % 2**32

In [3]:
dataset = np.random.randint(low=0,high=9999,size=(255,32))
w = 255
mix = np.random.randint(low=0,high=9999,size=(32*2,))
mixhashes = 4
n = mixhashes*12

In [4]:
dataset[0]

array([1473, 8840, 2326, 4862, 4376,  887, 4743, 1791, 8219, 7335,  263,
       9827, 9546, 9938, 8337, 1942, 9372, 4061, 2186, 5250, 6987, 5699,
       3379, 2987, 9747,  805,   57, 5341, 2605, 9086, 5538, 8911])

In [5]:
def dataset_lookup(i):
    global dataset
    return dataset[i%255]

In [6]:
fnv(i ^ 255, mix[i % 4]) % (n // mixhashes) * mixhashes

NameError: ignored

In [ ]:

mix = np.random.randint(low=0,high=9999,size=(32*2,))
for i in range(64):
    p = fnv(i ^ 255, mix[i % 4]) % (n // mixhashes) * mixhashes
    newdata = np.concatenate((dataset[p],dataset[p+1]))
    # print(p)
    # for j in range(2):
        # newdata.extend(dataset_lookup(p + j)))
    mix = fnv(mix,newdata)#list(map(fnv, mix, newdata))

In [ ]:
mix

array([3985079747, 1346983403, 1981568548, 1526319506, 2440194780,
       2757403039, 1238499430,  693978502, 1521365153, 1362517488,
        908417300, 2527320840, 1692219344, 4142246857, 2035723597,
       3673645746, 3694542588, 3350711610, 3920101748,  400326498,
       2441177567, 2542937078, 2537479304,  613273986,  380289098,
       3549577916, 2314743346, 1847837534, 2947075273, 3658211542,
       1901587902,  152269474, 3045458365, 2625950910, 2089250360,
       3732686165, 1743174506,  327231384, 2150551196, 1767836360,
       3289714304, 3780473959, 2292005251, 3141028232,  467145413,
       2603302246, 1649134025,  667225354, 2808328843,  630141566,
       2064213168, 1820392178,  934347014, 1585877621, 2148839595,
       3880023372, 1436153271,  365523700, 2813845128, 3962716669,
        856371812, 2521896176,  483345370, 3561152656])

In [ ]:
dataset[0]

array([1160, 1892, 5093, 8250, 1250, 4532, 2870, 4920, 7707, 4614, 2763,
       5595, 8953, 3948, 6445, 7570, 7391, 3969, 5648, 5623, 2919, 6589,
       9618,  611, 3237, 4851, 8215, 7484, 1904, 9287, 8098, 6465])

In [ ]:
p=0
np.concatenate((dataset[p],dataset[p+1]))

array([1160, 1892, 5093, 8250, 1250, 4532, 2870, 4920, 7707, 4614, 2763,
       5595, 8953, 3948, 6445, 7570, 7391, 3969, 5648, 5623, 2919, 6589,
       9618,  611, 3237, 4851, 8215, 7484, 1904, 9287, 8098, 6465, 1389,
       1191, 1714, 2925, 8183, 5855, 9590, 9294, 1595, 2251, 1473, 4376,
       5505, 1595,  978, 3816, 8507, 6548, 9773, 4088, 6597, 9379, 8745,
       1485, 5831, 5195, 7264, 3479, 3131, 6429,  300, 4291])